# Simple Web Crawler Implementation

A simple web crawler designed here is composed of 4 main modules:
* <b>Scheduler</b>: maintain a queue of URLs to visit
* <b>Downloader</b>: download web pages
* <b>Analyzer</b>: analyze content and links
* <b>Storage</b>: store content and metadata

## 1) Basic Downloader
Every web crawler should be defined a <i>name</i> and identified its <i>owner</i> (i.e., the '`user-agent`' and '`from`' fields of the headers, respectively). Sometimes, you may get an error message '`GET PAGE ERROR!`', caused by the connection timeout and the page not found, for instance. You can print '`r.status`' to track that problem.

In [1]:
import requests

headers = {
    'User-Agent': 'webcrawler-01',
    'From': 'thanwa.ji@ku.th'
}
seed_url = 'https://www.ku.ac.th/web2012/'

def get_page(url):
    global headers
    text = ''
    try:
        r = requests.get(url, headers=headers, timeout=2)
        text = r.text
    except(KeyboardInterrupt, SystemExit):
        raise
    except:
        print('GET PAGE ERROR!')
    return text.lower()

raw_html = get_page(seed_url)
print(raw_html)

<!doctype html public "-//w3c//dtd xhtml 1.0 transitional//en" "http://www.w3.org/tr/xhtml1/dtd/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta name="description" content="welcome to kasetsart university."/>
<meta name="keywords" content="มหาวิทยาลัย ,มก. ,มหาลัย ,ม , เกษตร, เกษตรศาสตร์, มหาวิทยาลัยเกษตร, ม.เกษตร, ม.เกษตรศาสตร์, ม.เกษตรฯ, นนทรี, ลูกนนทรี, วิทยาเขตบางเขน, วิทยาเขตกำแพงแสน, วิทยาเขตศรีราชา, วิทยาเขตเฉลิมพระเกียรติ จังหวัดสกลนคร,
 ตราประจำมหาวิทยาลัย, ดินแดนแห่งรอยยิ้ม, คณะเกษตร, คณะเศรษฐศาสตร์, คณะบริหารธุรกิจ, คณะสถาปัตยกรรมศาสตร์, คณะประมง, คณะสังคมศาสตร์, คณะมนุษยศาสตร์, คณะสัตวแพทยศาสตร์, คณะวนศาสตร์, คณะอุตสาหกรรมเกษตร, คณะวิทยาศาสตร์, คณะเทคนิคการสัตวแพทย์, คณะวิศวกรรมศาสตร์, คณะสิ่งแวดล้อม, คณะศึกษาศาสตร์,บัณฑิตวิทยาลัย, สำนักงานอธิการบดี, สถาบันวิจัยและพัฒนาแห่ง มก., สำนักทะเบียนและประมวลผล, สถาบันค้นคว้าและพัฒนาผลิตภัณฑ์อาหาร, สำนักบริการคอมพิวเตอร์, สถาบันค้นคว้า ,พัฒนาผลิตผลทางการเกษตร , อุตสาหกรรมเกษตร, สำนักพิพิธภัณฑ์ , วัฒ

## 2) Basic Analyzer
### 2.1 Link Parser
The following code is an example of simple link parser. The program extracts all links by considering the <i>anchor</i> tag only, and stores them into a `urls` list.

In [6]:
def link_parser(raw_html):
    urls = [];
    pattern_start = '<a href="';  pattern_end = '"'
    index = 0;  length = len(raw_html)
    while index < length:
        start = raw_html.find(pattern_start, index)
        if start > 0:
            start = start + len(pattern_start)
            end = raw_html.find(pattern_end, start)
            link = raw_html[start:end]
            if len(link) > 0:
                if link not in urls:
                    urls.append(link)
            index = end
        else:
            break
    return urls

### 2.2 URL Normalization
The following code is an example of using the `urljoin()` function to transform a relative URL to the absolute one.

In [5]:
from urllib.parse import urljoin

# Define an absolute (base) URL of a web page
base_url = 'https://mike.cpe.ku.ac.th'

# An example of the extracted absolute link
link_1 = 'http://www.ku.ac.th'
# An example of the extracted relative link
link_2 = 'download/homework.html'

# Resolve links
abs_link_1 = urljoin(base_url, link_1)
abs_link_2 = urljoin(base_url, link_2)

print(abs_link_1)  # -> http://www.ku.ac.th
print(abs_link_2)  # -> https://mike.cpe.ku.ac.th/download/homework.html

http://www.ku.ac.th
https://mike.cpe.ku.ac.th/download/homework.html


## 3) Basic Scheduler
The following code is an example of using a FIFO queue to handle the extracted URLs to be further downloaded. In particular, the main crawling process simply invokes the previous two defined functions, i.e., `get_page()` and `link_parser()`, to first download a web page and extract its out-going links, respectively. Then, all extracted links will be stored into a queue. We define here two queues: `frontier_q` and `visited_q`. The former is used as the FIFO queue to keep URLs for next downloading, while the latter is used to remember which web pages have been already downloaded.

In [9]:
seed_url = 'https://www.ku.ac.th/web2012/'
frontier_q = [seed_url]
visited_q = []

# @param 'links' is a list of extracted links to be stored in the queue
def enqueue(links):
    global frontier_q
    for link in links:
        if link not in frontier_q and link not in visited_q:
            frontier_q.append(link)

# FIFO queue
def dequeue():
    global frontier_q
    current_url = frontier_q[0]
    frontier_q = frontier_q[1:]
    return current_url

#--- main process ---#
current_url = dequeue()
visited_q.append(current_url)
raw_html = get_page(current_url)
extracted_links = link_parser(raw_html)
enqueue(extracted_links)

## 4) Storing Text into a File
As the following, we use the `os.makedirs()` function to first create (sub)directories. Notice that the `exist_ok=True` parameter is set to prevent an exception error if the target directory already exists. Then, we use the `open()`, `write()`, and `close()` functions to open a file, write some text into that file, and afterwards close it. In addition, we import the `codecs` module together with using the '`utf-8`' encoding for non-English content.

In [2]:
import os, codecs

# Create (sub)directories with the 0o755 permission
# @param 'exist_ok' is True for no exception if the target directory already exists
path = 'html/subdir1/subdir2'
os.makedirs(path, 0o755, exist_ok=True)

# Write content into a file
raw_html = '<html><body><a href="http://test1.com">test1</a><br><a href="http://test2.com">test2</a></body></html>'
abs_file = path + '/index.html'
f = codecs.open(abs_file, 'w', 'utf-8')
f.write(raw_html)
f.close()

# <font color="blue">Your Turn ...</font>
Write a web crawler to collect 10,000 web pages (including only '`.htm`' and '`.html`' files) within the '`ku.ac.th`' domain.